In [1]:
from datetime import datetime
from fund.fund import Fund
from pricer.pricer import Pricer as pricer_list
from ranker.ranker import Ranker as ranker_list
from classifier.classifier import Classifier as classifier_list
from portfolio.aportfolio import APortfolio
from parameters.parameters import Parameters 
import pandas as pd
from returns.products import Products
from processor.processor import Processor as p

In [2]:
# portfolio = APortfolio(pricer_list.QUARTERLY_STOCK_FINANCIAL
#                        ,classifier_list.QUARTERLY_STOCK_FINANCIAL_CLASSIFIER
#                        ,ranker_list.QUARTERLY_STOCK_EARNINGS_RANKER)
portfolio_ii = APortfolio(pricer_list.WEEKLY_STOCK_SPECULATION
                          ,classifier_list.WEEKLY_STOCK_SPECULATION_CLASSIFIER
                          ,ranker_list.WEEKLY_STOCK_ROLLING_RANKER)
# portfolio_iii = APortfolio(pricer_list.WEEKLY_STOCK_ROLLING
#                            ,classifier_list.WEEKLY_STOCK_SPECULATION_CLASSIFIER
#                           ,ranker_list.WEEKLY_STOCK_ROLLING_RANKER)
# portfolio_iv = APortfolio(pricer_list.WEEKLY_STOCK_WINDOW
#                           ,classifier_list.WEEKLY_STOCK_SPECULATION_CLASSIFIER
#                           ,ranker_list.WEEKLY_STOCK_ROLLING_RANKER)
portfolios = [portfolio_ii]

In [3]:
backtest_start_date = datetime(2023,1,1)
backtest_end_date  = datetime.now()
current_start_date = datetime(2023,1,1)

In [4]:
fund = Fund(portfolios,backtest_start_date,backtest_end_date,current_start_date)

In [5]:
fund.initialize_portfolios()
fund.initialize_backtesters()

In [8]:
test_portfolio = fund.portfolios[0]
risk_returns = test_portfolio.create_risk_returns()
sim = test_portfolio.create_current_simulation()
merged = test_portfolio.merge_sim_returns(sim,risk_returns)

In [18]:
merged

,year,week,ticker,price_prediction,rank_prediction,classification_prediction,close,high,low,open,...,quarter,risk_return_2,risk_return_3,risk_return_4,day,weekly_risk_return,bench_weekly_return,weekly_variance,market_weekly_cov,weekly_beta
0,2023,2,MMM,122.552902,120.771538,1.0,126.72,127.130,123.75,124.66,...,1.0,0.012545,0.034702,0.056704,4.0,0.056704,0.005322,0.000469,0.000139,0.296429
1,2023,2,AOS,58.700774,55.362971,1.0,61.44,61.680,59.85,60.05,...,1.0,0.012859,0.042770,0.073375,4.0,0.073375,0.005322,0.000469,0.000264,0.563017
2,2023,2,ABT,108.663675,102.858404,1.0,112.33,112.735,110.21,112.00,...,1.0,0.010071,0.025096,0.023135,4.0,0.023135,0.005322,0.000469,-0.000044,-0.093018
3,2023,2,ABBV,160.204013,145.692922,1.0,166.55,168.110,164.12,164.84,...,1.0,0.017472,0.025681,0.030567,4.0,0.030567,0.005322,0.000469,-0.000055,-0.117052
4,2023,2,ABMD,381.647428,NaN,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11216,2023,24,YUM,132.211526,131.573586,1.0,133.44,134.020,132.95,133.34,...,2.0,0.007551,0.004214,-0.005441,4.0,-0.005441,0.014693,0.000094,0.000056,0.595618
11217,2023,24,ZBRA,272.210026,294.525350,1.0,274.66,280.580,274.32,278.34,...,2.0,-0.010448,0.016883,0.020624,4.0,0.020624,0.014693,0.000094,0.000191,2.021172
11218,2023,24,ZBH,132.211526,129.178231,1.0,136.01,136.350,134.22,134.69,...,2.0,0.022094,0.044062,0.042462,4.0,0.042462,0.014693,0.000094,0.000087,0.917803
11219,2023,24,ZION,30.965639,36.999472,1.0,30.87,32.150,30.59,32.00,...,2.0,-0.019689,0.024900,0.074861,4.0,0.074861,0.014693,0.000094,0.000262,2.770332


In [9]:
parameter = {
    "value":True,
    "ceiling":True,
    "classification":False,
    "rank":False,
    "short":False,
    "market_return":1.5
}

In [10]:
stuff = test_portfolio.recommendation(merged,parameter)

In [20]:
rec = stuff[0]
trades = rec[rec["week"]==24].head(10)
trades

,year,week,ticker,weekly_delta,weekly_delta_sign,position,value,ceiling,classification,rank,short,market_return
21,2023,24,NVR,0.218059,1.0,0,True,True,False,False,False,1.5
43,2023,24,AVGO,0.039568,1.0,1,True,True,False,False,False,1.5
65,2023,24,CRM,0.038939,1.0,2,True,True,False,False,False,1.5
87,2023,24,ADBE,0.037247,1.0,3,True,True,False,False,False,1.5
109,2023,24,CTLT,0.036433,1.0,4,True,True,False,False,False,1.5
131,2023,24,CHD,0.032424,1.0,5,True,True,False,False,False,1.5
153,2023,24,FISV,0.027170,1.0,6,True,True,False,False,False,1.5
175,2023,24,TJX,0.026328,1.0,7,True,True,False,False,False,1.5
197,2023,24,MPWR,0.026312,1.0,8,True,True,False,False,False,1.5
219,2023,24,MCD,0.024456,1.0,9,True,True,False,False,False,1.5
